In [1]:
import torch
import torch.nn as nn
import os
import cv2
import numpy as np
import pandas as pd
import csv
import matplotlib.pyplot as plt
import warnings
from tqdm import tqdm
warnings.filterwarnings('ignore')

In [2]:
from torchvision import models
from torchvision import transforms
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix

In [3]:
if(os.path.split(os.getcwd())[1] == "Transfer_Learning"):
    os.chdir("..")
print("Current Working Directory: {}".format(os.path.split(os.getcwd())[1]))

cuda = True

Current Working Directory: progettoVIPM


In [4]:
from utils.loadersAndEnums import datasets
from utils.loadersAndEnums import networks
from utils.loadersAndEnums import ImageDataset

In [5]:
model = models.resnet50()
num_classes = 101
model.fc = nn.Linear(model.fc.in_features, num_classes)
model.load_state_dict(torch.load("C:\\Users\\marco\\Desktop\\Uni\\Magistrale\\VIPM\\Progetto\\resnet50model.pth")['model_state'])
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
model.to(device)

cuda


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [6]:
class ResNetFeatures(nn.Module):
    def __init__(self, original_model):
        super(ResNetFeatures, self).__init__()
        self.features = nn.Sequential(
            original_model.conv1,
            original_model.bn1,
            original_model.relu,
            original_model.maxpool,
            original_model.layer1,
            original_model.layer2,
            original_model.layer3,
            original_model.layer4[:-1]  # Exclude the last bottleneck
        )
        self.last_bottleneck = original_model.layer4[-1]  # Extract last bottleneck separately
        self.avgpool = original_model.avgpool

    def forward(self, x):
        x = self.features(x)
        x = self.last_bottleneck(x)  # Get features from the last bottleneck
        return x

In [8]:
feature_extractor = ResNetFeatures(model)

In [9]:
from torch.utils.data import DataLoader
trainData = datasets.TRAINING_LABELED
testData = datasets.TEST
train = ImageDataset(trainData,224,cuda=True)
test = ImageDataset(testData,224,cuda=True)
trainDL = DataLoader(train, batch_size=128, shuffle=True)
testDL = DataLoader(test, batch_size=128, shuffle=True)


In [10]:
feature_extractor.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
feature_extractor.to(device)
all_features_train = []
all_labels_train = []
with torch.no_grad():
    for images, labels in trainDL:
        images = images.to(device)
        labels = [int(label) for label in labels]
        labels = torch.tensor(labels, dtype=torch.long).to(device)
        features = feature_extractor(images)
        all_features_train.append(features.cpu().numpy())
        all_labels_train.append(labels.cpu().numpy())

In [11]:
feature_extractor.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
feature_extractor.to(device)
all_features_test = []
all_labels_test = []
with torch.no_grad():
    for images, labels in testDL:
        images = images.to(device)
        labels = [int(label) for label in labels]
        labels = torch.tensor(labels, dtype=torch.long).to(device)
        features = feature_extractor(images)
        all_features_test.append(features.cpu().numpy())
        all_labels_test.append(labels.cpu().numpy())

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_train = scaler.fit_transform(all_features_train)
scaled_test = scaler.transform(all_features_test)
pca = PCA(n_components=500).fit(scaled_train)
X_train_pca = pca.transform(scaled_train)
x_test_pca = pca.transform(scaled_test)
print("Final input shape train: {}".format(X_train_pca.shape))
print("Final input shape test: {}".format(x_test_pca.shape))